# Kernel methods

The regularized empirical risk minimization problem is : $\hat f = argmin_{f \in F} \frac{1}{n} \sum_{i = 1}^n L(y_i, f(x_i)) + \lambda \Omega (f)$

A simple example, linear models : $X = R^p$, $F = \{ f_w : x \mapsto w^T x | w \in R^p \}$, $\Omega (f_w) = ||w||^2_2$

By choosing carefully the loss function, we can create several well-known models :
- ridge regression : $L(y_i, w^T x_i) = \frac{1}{2} (y_i - w^T x_i)^2$
- linear SVM : $L(y_i, w^T x_i) = max(0, 1 - y_i w^T x_i)$
- logistic regression : $L(y_i, w^T x_i) = log(1 + e^{-y_i w^T x_i})$

Unfortunately, linear models often perform poorly unless the problem features are well-engineered or the problem is very simple.

To solve this problem, we need to change the functional space F :  
1. By choosing F as a deep learning space  
2. By choosing F as a RKHS

## I) Kernels and RKHS

### 1) Positive definite kernel

The kernel method is based on pairwise comparisons between data points. We define a "comparison function" $K : X^2 \to R$ and represent a set of n data points $S = { x_1, ... x_n}$ by the n x n matrix $K = [K(x_i, x_j)]_{1 <= i,j <= n}$. However, we will restrict ourselves to a particular class of pairwise comparison functions : positive definite kernels.

**Definition :** A **positive definite kernel** on the set X is a function $K : X^2 \to R$ that is symmetric and which satisfies : $$\forall n \in N, \forall x_1, ... x_n \in X^n, \forall a_1, ... a_n \in R^n, \sum_i \sum_j a_i a_j K(x_i, x_j) \geq 0$$

Equivalently, a kernel K is pd if and only if for any set of n data points, the associated matrix K is symmetric and positive semidefinite.

$\forall n \in N, \forall x_1, ... x_n \in X^n, K = [K(x_i, x_j)]_{1 <= i,j <= n}$ is symmetric and positive semidefinite, ie :
$$K^T = K$$
$$\forall u \in R^n, u^T K u \geq 0$$

> __Example : linear kernel__  
$X = R^d$
$$K : X^2 \to R \\
(x, y) \mapsto \langle x, y \rangle$$
K is symmetric by definition of the inner product in $R^d$ and verifies : $\sum_i \sum_j a_i a_j K(x_i, x_j) = \sum_i \sum_j a_i a_j \langle x_i, x_j \rangle = \langle \sum_i a_i x_i, \sum_j a_j x_j \rangle = || \sum_i a_i x_i ||^2 \geq 0$

**Lemma :** $\phi : X \to R^d$. If 
$$K : X^2 \to R \\
(x, y) \mapsto \langle \phi(x), \phi(y) \rangle$$
Then K is a pd kernel.

> **Proof :** K is symmetric by definition of the inner product in $R^d$ and verifies : $\sum_i \sum_j a_i a_j K(x_i, x_j) = \sum_i \sum_j a_i a_j \langle \phi(x_i), \phi(x_j) \rangle = \langle \sum_i a_i \phi(x_i), \sum_j a_j \phi(x_j) \rangle = || \sum_i a_i \phi(x_i) ||^2 \geq 0$


> __Example : polynomial kernel__  
$X = R^2$
$$K(x, y) = \langle \phi(x), \phi(y) \rangle$$
where $\phi : R^2 \to R^3 \\ x = (x_1, x_2) \mapsto (x_1^2, \sqrt2x_1x_2, x_2^2)$  
Then K is a pd kernel and we can show that :  
$$K(x, y) = x_1^2 y_1^2 + 2x_1x_2y_1y_2 + x_2^2y_2^2 = (x_1y_1 + x_2y_2)^2 = \langle x, y \rangle ^2$$

The converse of the previous lemma is a fundamental theorem in kernel methods : it shows that any pd kernels can be considered as an inner product in a Hilbert space.

**Theorem : K is a pd kernel if and only if there exists a Hilbert space H and a mapping $\phi : X \to H$ such that 
$$\forall x, y \in H^2, K(x, y) = \langle \phi(x), \phi(y) \rangle$$**

> **Proof :** finite case  
($\Leftarrow$) Already proved.  
($\Rightarrow$) Assume $X = {x_1, ... x_N}$ is finite of size N.  
Any positive definite kernel $K : X \times X \to R$ is entirely defined by the $N \times N$ symmetric positive semidefinite matrix $K = [(K(x_i, x_j)]_{i, j}$.  
It can therefore be diagonalized (why?) on an orthonormal basis of eigenvectors $(u_1, ... u_N)$, with non-negative eigenvalues $0 \leq \lambda_1 \leq ... \leq \lambda_N$, i.e.,
$$K_{i, j} = K(x_i, x_j) = [\sum_{k=1}^N \lambda_k u_k u_k^T]_{i,j} = \sum_{k=1}^N \lambda_k u_{ik} u_{jk} = \langle \phi(x_i), \phi(x_j) \rangle$$
with
$$\phi(x_i) = (\sqrt{\lambda_1} u_{i1}, ... \sqrt{\lambda_N}u_{iN})$$

> **Proofs :**
- if X is a compact and K continuous : Mercer's proof
- if X is countable : Kolmogorov's proof
- for the general case : Aronszajn's proof

We will go through the proof of the general case by introducing the concept of Reproducing Kernel Hilbert Space (RKHS).

### 2) Reproducing Kernel Hilbert Space (RKHS)

**Definition :**  Let X be a set, $H \subset R^X$ a class of functions $X \to R$ forming a (real) Hilbert space (with inner product $\langle ., . \rangle$).  
The function $K : X^2 \to R$ is called a **reproducing kernel** of the Hilbert space H if and only if :
- H contains all functions of the form : $\forall x \in X, K_x : t \to K(x, t)$
- For all $x \in X$ and for all $f \in H$, the reproducing property holds : $f(x) = \langle f, K_x \rangle$

If a reproducing kernel of H exists, then H is called a **reproducing kernel Hilbert space** (RKHS).

**Theorem : The Hilbert space H is a RKHS if and only if for all $x \in X$ $$F : H \to R \\ f \mapsto f(x)$$ is continuous.**

> **Proof :**  
($\Rightarrow$) H is a RKHS. We wonder if $$F : H \to R \\ f \mapsto f(x)$$ is continuous  
We can show that F is L-smooth. Because H is a RKHS, there exists a reproducing kernel K and for any $x \in X$ and any $f, g \in H^2$ :
$$ || F(f) - F(g) || = | f(x) - g(x) | = 
| \langle f - g, K_x \rangle | \\\leq || f - g ||_H . || K_x ||_H \leq || f - g ||_H . \sqrt \langle K_x, K_x \rangle \\\leq || f - g ||_H . \sqrt{K(x, x)}$$
Hence, F is L-smooth (with $L = \sqrt{K(x, x)}$) and thus continuous.  
($\Leftarrow$) F is continuous. We want to show that H is a RKHS, i.e. there exists a reproducing kernel K for H  
By using the **Riesz representation theorem** (an important property of Hilbert spaces) : if H is an Hilbert space then any continuous linear form f on H can be written as the inner product such that $f(.) = \langle ., y \rangle$ where $y \in H$ is unique  
Yet, F is a continuous linear form on H where the elements of H are functions. Hence :
$$ \forall x \in X, \exists ! g_x \in H, F(f) = f(x) = \langle f, g_x \rangle$$
Finally, the function $K(x, y) = g_x (y)$ is a rk for H because it holds the reproducing property and $\forall x \in X, g_x \in H$.

**Corollary :** Convergence in a RKHS implies pointwise convergence, i.e. if $(f_n)_{n \in N}$ converges to $f$ in H, then, for any $x \in X$, $(f_n(x))_{n \in N}$ converges to $f(x)$ .

**Theorem :  
\- If H is a RKHS, then it has a unique reproducing kernel  
\- Conversely, a function K can be the reproducing kernel of at most one RKHS**

The following theorem proves the equivalence between a positive definite kernel and a reproducing kernel and will allow us to prove the fundamental theorem which says that any positive definite kernel can be represented as an inner product in some Hilbert space.

**Theorem : A function $K : X^2 \to R$ is a positive definite kernel if and only if it is a reproducing kernel for some Hilbert space H.**

> **Proof :**  
>($\Leftarrow$) If K is a reproducing kernel for a Hilbert space H, then it can be expressed as :
$$K(x, y) = K_x(y) = \langle K_x, K_y \rangle$$
Hence, K is symmetric by definition of the inner product in H and
$$\forall x_1, ... x_n \in X^n, \forall a_1, ... a_n \in R, 
\\\sum_i \sum_j a_i a_j K(x_i, x_j) 
\\= \langle \sum_i a_i K_{x_i}, \sum_j a_j K_{x_j} \rangle 
\\= || \sum_i a_i K_{x_i} || ^2 \geq 0$$
Then, K is a positive definite kernel.  
>($\Rightarrow$) K is a positive definite kernel. We need to create a RKHS H for which K will be the reproducing kernel.  
Let $H_0$ be the vector subspace of $R^X$ spanned by the functions $(K_x)_{x \in X}$ :
$$H_0 = vect((K(x, .))_{x \in X})$$   
We want to define an inner product such that $H_0$ is an pre-Hilbert space.  
For any $f,g \in H_0^2$, given by $f = \sum_{i=1}^m a_i K_{x_i}$, $g = \sum_{j=1}^n b_j K_{x_j}$, let :
$$\langle f,g \rangle _{H_0} = \sum_i \sum_j a_i b_j K(x_i, x_j)$$
We can show that $(H_0, \langle .,. \rangle)$ is a pre-Hilbert space and K verifies the reproducing kernel conditions.  
We can observe that any Cauchy sequences of $H_0$ converges to f, which is not necessarily in $H_0$. Thus, we don't have the completeness, so $H_0$ is not an Hilbert space. We then extends $H_0$ by creating $H \subset R^X$ to be the set of functions $f : X \to R$ which are pointwise limits of Cauchy sequences (of functions) of $H_0$.  
We can observe that $H_0 \subset H$ by taking the following sequences $(f_n = f)_{n\in N} \in H_0^N$ which converges pointwise to any $f \in H_0$.  
We define the following inner product in H :
$$\langle f,g \rangle_H = lim_{n \to \infty} \langle f_n, g_n \rangle$$
We can observe that this limit exists and is unique. What's more, it is easy to see that $\langle .,. \rangle_H$ is an inner product, using the same properties of $\langle .,. \rangle_{H_0}$.  
By construction, we can show that H is complete and that K is a reproducing kernel for H (in particular, the reproducing property holds).

Finally, we can deduce easily the Aronszajn's theorem (the general case of Mercer's theorem).

**Theorem : Aronszajn's theorem  
K is a positive definite kernel on the set X if and only if there exists a Hilbert space H and a mapping $\phi : X \to H$ such that, for any $x, y \in X^2$ :
$$ K(x, y) = \langle \phi(x), \phi(y) \rangle _H $$**

> **Proof :**  
($\Leftarrow$) Already proved.  
($\Rightarrow$) We proved that if K is a positive definite kernel then there exists a Hilbert space H such that K is a reproducing kernel for H. If we define the mapping $\phi : X \to H$ by :
$$ \forall x \in X, \phi(x) = K_x = K(x, .)$$
Then, by reproducing property, we have :
$$ \forall (x, y) \in X^2, \langle \phi(x), \phi(y) \rangle _H = \langle K_x, K_y \rangle _H = K_x(y) = K(x, y)$$

### 3) My first kernels

Let's see some kernel examples and discover the RKHS associated to these kernels.

### 4) Smoothness functional

There is a natural way to regularize functions in a RKHS. Indeed, by Cauchy-Schwarz we have, for any $f \in H$ and any two points $x,y \in X$ :
$$ |f(x) - f(y)| = | \langle f, K_x - K_y \rangle_H | \leq || f ||_H . || K_x - K_y ||_H = || f ||_H . d_K (x, y) $$

The norm of a function in the RKHS controls **how fast** the function varies over X with respect to **the geometry defined by the kernel** (smooth with constant $‖f‖_H$).

### 5) The kernel trick

We can show that kernel methods allow us to create efficient nonlinear methods.

## II) Kernel methods : supervised learning

**Regularized empirical risk formulation :**  
The goal is to learn a **prediction function** $f : X \to Y$ given labeled training data $(x_i \in X, y_i \in Y)_{1 \leq i \leq n}$ :
$$min_{f \in H} \frac{1}{n} \sum_{i=1}^n L(y_i, f(x_i)) + \lambda ||f||_H^2$$.

What are the new perspectives with kernel methods ?
- being able to deal with non-linear functional spaces endowed with a natural regularization function $||.||^2_H$
- being able to deal with non-vectorial data (graphs, tress)

Two theoretical results underpin a family of powerful algorithms for data analysis using positive definite kernels, collectively known as kernel methods:
- The **kernel trick**, based on the representation of positive definite kernels as inner products
- The **representer theorem**, based on some properties of the regularization functional defined by the RKHS norm

### 1) The representer theorem

The representer theorem says that the solution to a regularized empirical risk minimization problem in a RKHS lives in the vector subspace spanned by the kernel functions, which is a concrete optimization problem in $R^n$.

**Theorem : the representer theorem  
Let X be a set endowed with a positive definite kernel K, $H_K$ the corresponding RKHS, and $S = \{x_1, ... x_n\} \subset X$ a finite set of points in X. Let $\Psi : R^{n+1} \to R$ be a function of n+1 variables, strictly increasing with respect to the last variable.  
Then, any solution to the optimization problem :
$$min_{f \in H_K} \Psi(f(x_1), ... f(x_n), ||f||_{H_K})$$
admits a representation of the form :
$$\forall x \in X, f(x) = \sum_{i=1}^n \alpha_i K(x_i, x)$$**

> **Proof :**
Let V be the linear space spanned by $(K(x_i, .))_{1 \leq i \leq n}$. 
$$ V = \{ f \in H_K | f(x) = \sum_{i=1}^n \alpha_i K(x_i, x), (\alpha_1, ... \alpha_n) \in R^n \} $$
V is a finite-dimensional subspace of H, therefore any function $f \in H_K$ can be uniquely decomposed as : $f = f_V + f_{V^T}$ with $f_V \in V$ and $f_{V^T} \in V^T$. Since $H_K$ is a RKHS with kernel K, for $1 \leq i \leq n$,
$f_{V^T} (x_i) = \langle f, K_{x_i} \rangle$ and $f(x_i) = \langle f, K_{x_i} \rangle = f_V (x_i)$.  
Pythagora's theorem in $H_K$ then shows that : $||f||_{H_K}^2 = ||f_V||_{H_K}^2 + ||f_{V^T}||_{H_K}^2$.  
Let $\epsilon (f)$ be the function which is minimized in the statement of the representer theorem. As a consequence,
$$\epsilon (f) \geq \epsilon (f_V)$$
which is equality if and only if $||f_{V^T}||=0$.  
**The minimum of $\Psi$ is therefore necessarily in V.**

Often the function $\Psi$ has the form :
$$ \Psi(f(x_1), ... f(x_n), ||f||_{H_K}) = c(f(x_1), ... f(x_n)) + \lambda \Omega(||f||_{H_K})$$
where $c(.)$ measures the "fit" of f to a given problem (regression, classification, dimension reduction, ...) and $\Omega$ is strictly increasing.

This formulation has two important consequences :
- **Theoretically**, the minimization will enforce the norm $‖f‖_{H_K}$ to be "small", which can be beneficial by ensuring a sufficient level of smoothness for the solution (regularization effect).
- **Practically**, we know by the representer theorem that the solution lives in a subspace of dimension n, which can lead to efficient algorithms although the RKHS itself can be of infinite dimension.

Most kernel methods have two complementary interpretations :
- A **geometric interpretation** in the feature space, thanks to the kernel trick. Even when the feature space is “large”, most kernel methods work in the linear span of the embeddings of the points available.
- A **functional interpretation**, often as an optimization problem over (subsets of) the RKHS associated to the kernel.

### 2) Kernel ridge regression

Kernel ridge regression is a useful extension of ridge regression by searching a solution function in a RKHS. This extension is allowed thanks to the kernel trick. By the representer theorem, the solution can be easily written and compute.

### 3) Classification with empirical risk minimization

optimization in RKHS can be seen as an unconstrained and convex optimization problem in $R^n$

### 4) A (tiny) bit of learning theory

definitions

large-margin classifiers -> empirical risk minimization

capacity and Rademacher complexity

basic learning bounds

ERM in RKHS balls, constrained optimization problem

### 5) Foundations of constrained optimization

convexity, Lagrangian, duality, KKT conditions, Slater's condition

### 6) Support vector machines

## III) Kernel methods : unsupervised learning

## IV) The kernel jungle

## V) Open problems and research topics

Bibliographie :

http://lear.inrialpes.fr/people/mairal/teaching/2015-2016/MVA/fichiers/mva_slides.pdf